## Complete Market Models & Pricing Library ##

# Abstract

Risk Neutral Valuation has profoundly influenced the field of mathematical finance. The pioneering work of Bachelier laid the groundwork for subsequent researchers, including Black, Scholes, and Merton, whose contributions have shaped modern financial theory. Today, these mathematical foundations are integral to the models used for pricing derivatives on the trading floors of banks, asset management firms, and other financial institutions. Concurrently, Python has emerged as the most prevalent language for implementing these models. This paper demonstrates how to build pricers for both vanilla and exotic options using object-oriented programming in Python. The classes developed in this paper leverage Monte Carlo simulation and geometric Brownian motion as the underlying stochastic processes to provide accurate and efficient pricing solutions.


### Introduction

Risk Neutral Valuation marks a significant milestone in mathematical finance, tracing back to Bachelier's pioneering work, and subsequently refined by Black, Scholes, and Merton. This approach underpins the pricing models for derivatives widely used across financial institutions. With Python's growing popularity in the financial sector, this project showcases the implementation of option pricers using object-oriented programming.

### Methodology

#### Geometric Brownian Motion

Geometric Brownian Motion (GBM) is the stochastic process used to model the underlying asset's price dynamics.

#### Monte Carlo Simulation

Monte Carlo Simulation is a computational algorithm that uses repeated random sampling to obtain numerical results, often used in option pricing to simulate the various paths an asset price might take.


Discounting Class:

The class models risk-neutral discounting, which is a fundamental concept in option pricing.
The risk-neutral discount factor is given by 
𝐷
(
𝑡
)
=
𝑒
−
𝑟
𝑡
D(t)=e 
−rt
 , where 
𝑟
r is the risk-free short rate, and 
𝑡
t is the time to maturity. This formula is used to discount future cash flows to their present value under the risk-neutral measure.

In [44]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

# Risk-neutral discounting class
class Discounting:
    def __init__(self, short_rate):
        self.short_rate = short_rate

    def discount_factor(self, t):
        return np.exp(-self.short_rate * t)

EuropeanOptionParameters Class:

This class encapsulates the parameters necessary for modeling a European option, including:
Initial value 
𝑆
0
S 
0
​
 : The starting price of the underlying asset.
Volatility 
𝜎
σ: The standard deviation of the asset's returns, representing the uncertainty or risk associated with the asset.
Time to maturity 
𝑇
T: The duration until the option expires.
Strike price 
𝐾
K: The predetermined price at which the option can be exercised.
Option type (call or put): Defines whether the option gives the right to buy (call) or sell (put) the underlying asset.
Dividends 
𝑞
q: The continuous dividend yield of the underlying asset.


In [45]:
# Class to model the parameters of the underlying risk factor and European option
class EuropeanOptionParameters:
    def __init__(self, initial_value, volatility, time_to_maturity, strike_price, option_type, dividends=0):
        self.initial_value = initial_value
        self.volatility = volatility
        self.time_to_maturity = time_to_maturity
        self.strike_price = strike_price
        self.option_type = option_type
        self.dividends = dividends

EuropeanVanillaOption Class:

This class uses Monte Carlo simulation to price European vanilla options. The geometric Brownian motion (GBM) model is used to simulate the paths of the underlying asset.

In [46]:
# Class to value European vanilla options using Monte Carlo simulation
class EuropeanVanillaOption:
    def __init__(self, parameters, discounting):
        self.params = parameters
        self.discounting = discounting

    def generate_asset_prices(self, n_simulations, n_steps):
        dt = self.params.time_to_maturity / n_steps
        prices = np.zeros((n_simulations, n_steps + 1))
        prices[:, 0] = self.params.initial_value
        for t in range(1, n_steps + 1):
            z = np.random.standard_normal(n_simulations)
            prices[:, t] = prices[:, t - 1] * np.exp((self.params.dividends - 0.5 * self.params.volatility ** 2) * dt + self.params.volatility * np.sqrt(dt) * z)
        return prices

    def price(self, n_simulations=10000, n_steps=100):
        prices = self.generate_asset_prices(n_simulations, n_steps)
        if self.params.option_type == 'call':
            payoffs = np.maximum(prices[:, -1] - self.params.strike_price, 0)
        else:
            payoffs = np.maximum(self.params.strike_price - prices[:, -1], 0)
        discounted_payoff = np.mean(payoffs) * self.discounting.discount_factor(self.params.time_to_maturity)
        return discounted_payoff

The GBM model is defined by the stochastic differential equation:
𝑑
𝑆𝑡
=
(
𝑟
−
𝑞
)
𝑆
𝑡
 
𝑑
𝑡
+
𝜎
𝑆
𝑡
 
𝑑
𝑊
𝑡
dS 
t
​
 =(r−q)S 
t
​
 dt+σS 
t
​
 dW 
t
​
 
where 
𝑆
𝑡
S 
t
​
  is the asset price at time 
𝑡
t, 
𝑟
r is the risk-free rate, 
𝑞
q is the dividend yield, 
𝜎
σ is the volatility, and 
𝑊
𝑡
W 
t
​
  is a Wiener process (standard Brownian motion).


The Monte Carlo simulation generates multiple paths for the asset price and calculates the payoff for each path. The option price is the discounted average of these payoffs.

EuropeanLookbackOption Class:

This class prices European lookback options, which are exotic options where the payoff depends on the minimum or maximum price of the underlying asset during the option's life.
For a floating strike lookback call, the payoff is 
𝑆
𝑇
−
min
⁡
(
𝑆
𝑡
)
S 
T
​
 −min(S 
t
​
 ), where 
𝑆
𝑇
S 
T
​
  is the final asset price and 
min
⁡
(
𝑆
𝑡
)
min(S 
t
​
 ) is the minimum asset price during the life of the option.
For a floating strike lookback put, the payoff is 
max
⁡
(
𝑆
𝑡
)
−
𝑆
𝑇
max(S 
t
​
 )−S 
T
​
 .
For a fixed strike lookback option, the payoff is based on the maximum or minimum price relative to a fixed strike price.

In [47]:
# Class to value European lookback options
class EuropeanLookbackOption:
    def __init__(self, parameters, discounting, strike_type='floating'):
        self.params = parameters
        self.discounting = discounting
        self.strike_type = strike_type

    def generate_asset_prices(self, n_simulations, n_steps):
        dt = self.params.time_to_maturity / n_steps
        prices = np.zeros((n_simulations, n_steps + 1))
        prices[:, 0] = self.params.initial_value
        for t in range(1, n_steps + 1):
            z = np.random.standard_normal(n_simulations)
            prices[:, t] = prices[:, t - 1] * np.exp((self.params.dividends - 0.5 * self.params.volatility ** 2) * dt + self.params.volatility * np.sqrt(dt) * z)
        return prices

    def price(self, n_simulations=10000, n_steps=100):
        prices = self.generate_asset_prices(n_simulations, n_steps)
        if self.strike_type == 'floating':
            if self.params.option_type == 'call':
                payoffs = prices[:, -1] - np.min(prices, axis=1)
            else:
                payoffs = np.max(prices, axis=1) - prices[:, -1]
        else:
            if self.params.option_type == 'call':
                payoffs = np.maximum(np.max(prices, axis=1) - self.params.strike_price, 0)
            else:
                payoffs = np.maximum(self.params.strike_price - np.min(prices, axis=1), 0)
        discounted_payoff = np.mean(payoffs) * self.discounting.discount_factor(self.params.time_to_maturity)
        return discounted_payoff

EuropeanAsianOption Class:

This class prices European Asian options, which are options where the payoff depends on the average price of the underlying asset over a certain period.
For a fixed strike Asian call, the payoff is 
max
⁡
(
𝐴
−
𝐾
,
0
)
max(A−K,0), where 
𝐴
A is the average price of the asset and 
𝐾
K is the strike price.
For a floating strike Asian call, the payoff is 
max
⁡
(
𝑆
𝑇
−
𝐴
,
0
)
max(S 
T
​
 −A,0).
The Monte Carlo simulation generates multiple paths for the asset price, calculates the average price for each path, and computes the payoff. The option price is the discounted average of these payoffs.

In [48]:
# Class to value European Asian options
class EuropeanAsianOption:
    def __init__(self, parameters, discounting, strike_type='fixed'):
        self.params = parameters
        self.discounting = discounting
        self.strike_type = strike_type

    def generate_asset_prices(self, n_simulations, n_steps):
        dt = self.params.time_to_maturity / n_steps
        prices = np.zeros((n_simulations, n_steps + 1))
        prices[:, 0] = self.params.initial_value
        for t in range(1, n_steps + 1):
            z = np.random.standard_normal(n_simulations)
            prices[:, t] = prices[:, t - 1] * np.exp((self.params.dividends - 0.5 * self.params.volatility ** 2) * dt + self.params.volatility * np.sqrt(dt) * z)
        return prices

    def price(self, n_simulations=10000, n_steps=100):
        prices = self.generate_asset_prices(n_simulations, n_steps)
        average_prices = np.mean(prices, axis=1)
        if self.strike_type == 'fixed':
            if self.params.option_type == 'call':
                payoffs = np.maximum(average_prices - self.params.strike_price, 0)
            else:
                payoffs = np.maximum(self.params.strike_price - average_prices, 0)
        else:
            if self.params.option_type == 'call':
                payoffs = np.maximum(prices[:, -1] - average_prices, 0)
            else:
                payoffs = np.maximum(average_prices - prices[:, -1], 0)
        discounted_payoff = np.mean(payoffs) * self.discounting.discount_factor(self.params.time_to_maturity)
        return discounted_payoff

This project utilizes hypothetical values for demonstration purposes, meaning they are not actual market quotes from Bloomberg or Eikon Reuters. The primary aim is to illustrate that the pricing engine for each option type is functioning correctly.

For future work, it would be beneficial to incorporate real market data and perform a comparative analysis with established financial software such as Murex, Calypso, or Bloomberg. Additionally, benchmarking the results against QuantLib, an open-source library for quantitative finance, would provide further validation of the pricing models. This approach would not only enhance the credibility of the library but also ensure its practical applicability in real-world financial environments.

In [49]:
# Example Usage
# Parameters for the options
initial_value = 100
volatility = 0.2
time_to_maturity = 1
strike_price = 100
option_type = 'call'
short_rate = 0.05
dividends = 0.02

In [50]:
# Create instances of the classes
discounting = Discounting(short_rate)
parameters = EuropeanOptionParameters(initial_value, volatility, time_to_maturity, strike_price, option_type, dividends)

In [51]:
# European Vanilla Option
vanilla_option = EuropeanVanillaOption(parameters, discounting)
vanilla_price = vanilla_option.price()
print(f"European Vanilla Option Price: {vanilla_price}")

European Vanilla Option Price: 8.68700442619533


In [52]:
# European Lookback Option (Floating Strike)
lookback_option_floating = EuropeanLookbackOption(parameters, discounting, strike_type='floating')
lookback_price_floating = lookback_option_floating.price()
print(f"European Lookback Option Price (Floating Strike): {lookback_price_floating}")

European Lookback Option Price (Floating Strike): 14.525730537360635


In [53]:
# European Lookback Option (Fixed Strike)
lookback_option_fixed = EuropeanLookbackOption(parameters, discounting, strike_type='fixed')
lookback_price_fixed = lookback_option_fixed.price()
print(f"European Lookback Option Price (Fixed Strike): {lookback_price_fixed}")

European Lookback Option Price (Fixed Strike): 15.854392894769227


In [54]:
# European Asian Option (Fixed Strike)
asian_option_fixed = EuropeanAsianOption(parameters, discounting, strike_type='fixed')
asian_price_fixed = asian_option_fixed.price()
print(f"European Asian Option Price (Fixed Strike): {asian_price_fixed}")

European Asian Option Price (Fixed Strike): 4.90008726365355


In [55]:
# European Asian Option (Floating Strike)
asian_option_floating = EuropeanAsianOption(parameters, discounting, strike_type='floating')
asian_price_floating = asian_option_floating.price()
print(f"European Asian Option Price (Floating Strike): {asian_price_floating}")

European Asian Option Price (Floating Strike): 4.83085428810326


Conclusion
This project presents a comprehensive pricing library for European plain vanilla and exotic options using Monte Carlo simulation and geometric Brownian motion as the stochastic process. The key components of the library include classes for risk-neutral discounting, modeling option parameters, and pricing various types of options such as European vanilla, lookback, and Asian options.

Key Contributions
Risk-Neutral Discounting:

The library incorporates a robust risk-neutral discounting mechanism. This fundamental concept is essential for present value calculations of future cash flows under the risk-neutral measure, ensuring that the pricing models adhere to the no-arbitrage condition.
Parameter Modeling:

The library provides a flexible framework for defining the parameters of the underlying asset and options. This includes initial asset value, volatility, time-to-maturity, strike price, option type (call or put), and dividends. Such flexibility allows the library to accommodate a wide range of market scenarios and financial instruments.
Monte Carlo Simulation:

At the core of the library is the Monte Carlo simulation technique, which is used to model the stochastic behavior of asset prices. By simulating numerous paths of the asset price under geometric Brownian motion, the library can accurately estimate the expected payoff of various options.
The use of Monte Carlo simulation allows for the pricing of options with complex features and path dependencies, such as Asian and lookback options, which are difficult to price using closed-form analytical solutions.
Geometric Brownian Motion:

The library employs the geometric Brownian motion model to describe the dynamics of the underlying asset price. This model captures the essential characteristics of asset price movements, including the drift (representing the expected return) and volatility (representing the risk or uncertainty).
The stochastic differential equation underlying geometric Brownian motion ensures that the simulated asset prices are consistent with the theoretical assumptions of continuous-time finance.
Pricing Exotic Options:

The library extends the pricing capabilities to exotic options such as lookback and Asian options. Lookback options, which depend on the maximum or minimum asset price during the option's life, provide valuable insights for investors seeking to hedge against extreme market movements.
Asian options, which depend on the average asset price, are particularly useful in markets where the underlying asset is subject to significant price fluctuations. The library's ability to price these options enhances its applicability to a broader range of financial instruments.
Practical Implications
The developed pricing library has significant practical implications for financial institutions, asset managers, and risk managers. By providing a versatile and accurate tool for option pricing, the library facilitates better decision-making and risk management. The use of Monte Carlo simulation and geometric Brownian motion ensures that the pricing models are robust and reflective of real-world market dynamics.

Future Enhancements
While the current version of the library offers a solid foundation for option pricing, several enhancements could further improve its functionality:

Parallel Computing: Implementing parallel computing techniques could significantly reduce the computational time required for Monte Carlo simulations, especially for large-scale simulations with high accuracy requirements.

Advanced Stochastic Models: Incorporating advanced stochastic models, such as stochastic volatility models (e.g., Heston model) or jump-diffusion models, could provide more accurate pricing for options in markets with volatility clustering or sudden price jumps.

Calibration and Validation: Developing methods for calibrating the model parameters to market data and validating the pricing models against observed option prices could enhance the accuracy and reliability of the library.

User Interface: Creating a user-friendly interface or integrating the library with financial software platforms could make it more accessible to practitioners in the finance industry.

Conclusion
This project demonstrates the successful development of a pricing library for European plain vanilla and exotic options using Monte Carlo simulation and geometric Brownian motion. The library's flexibility, accuracy, and robustness make it a valuable tool for option pricing and risk management. By addressing the practical needs of financial professionals and offering opportunities for future enhancements, this project contributes to the advancement of quantitative finance and financial engineering.